# Prototype Development

In [1]:
%load_ext autoreload
%autoreload 2

from melbviz.prototype import PedestrianDemo
from melbviz.config import COUNTS_PATH, SENSOR_PATH, ROOT_PATH

In [2]:
demo = PedestrianDemo.load(COUNTS_PATH, sensor_csv_path=SENSOR_PATH)

In [5]:
# TODO
# -- deploy using voila
# -- address artefcts in plots
# -- add prev 4 week average to time series plots etc

demo.prototype()

## Comparing Sensors by Year

In [5]:
import functools

# inputs
sensor_input = widgets.Dropdown(options=SENSORS)
year_input = widgets.Dropdown(options=YEARS)
month_input = widgets.Dropdown(options=MONTHS)

# binding inputs to output function
out = widgets.interactive_output(
    display_output(functools.partial(plot_years, foot_df)),
    {"sensor": sensor_input, "month": month_input},
)

# layout
widgets.HBox(
    [
        widgets.VBox(
            [
                widgets.HTML("<H1>Comparing Sensors by Year</h1>"),
                sensor_input,
                month_input,
            ]
        ),
        out,
    ]
)

NameError: name 'widgets' is not defined

Ideas:
* align to eg first Monday of the month
* get rid of the 2000 from the initial x value see `tickformatstops` example near bottom of [this page](https://plotly.com/python/time-series/)
* extend to `sensor_plot`
* add ability to overlay mean/median lines from custom date (ranges)
* work out how to combine `plot_sensors` and `plot_year` into single `plot_comparison` that also allows selecting month, and even day?
* improve hover label

## Ipywidgets

* input controls
* display outputs
* binding inputs and outputs to interactions

In [ ]:
import ipywidgets as widgets

In [ ]:
widgets.Dropdown(options=["cat", "dog"])

In [ ]:
widgets.Dropdown(options=[1,2,3,4, 5])

In [ ]:
widgets.IntRangeSlider(
    value=[2009, 2020],
    min=2009,
    max=2020,
)